<a href="https://colab.research.google.com/github/beatlesatani/coding_september/blob/main/web_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
pip install flask-ngrok

In [39]:
!pip install tensorflow_text
!pip install transformers

In [ ]:
from transformers import BertTokenizer,TFBertModel,BertConfig,TFBertForSequenceClassification
import tensorflow as tf
import torch
import numpy as np
from tensorflow import keras


In [40]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [41]:
model_path = '/drive/MyDrive/sentiment_analysis/bert_model.tf'
# load the model
loaded_model = keras.models.load_model(model_path)


In [193]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased",do_lower_case = True)

In [203]:
def mask_inputs_for_bert(tweets, max_len):
  # Encode the tweet using the tokenizer.
  encoded_dict = tokenizer.encode_plus(
            tweets,
            add_special_tokens=True, #add cls and sep
            max_length=max_len,   #pad & truncate all sentences
            pad_to_max_length=True,
            return_attention_mask=True  #construct attention_masks
        )
  input_id = encoded_dict['input_ids']
  attention_mask =encoded_dict['attention_mask']

  # Convert the lists to tensors (vectors) and return
  input_id = tf.convert_to_tensor(input_id)
  attention_mask = tf.convert_to_tensor(attention_mask)

  return input_id, attention_mask

In [244]:
test_sentence = "how's it going"
inp,mask = mask_inputs_for_bert(test_sentence,75)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [245]:
inp = tf.reshape(inp, (1, -1))
mask = tf.reshape(mask, (1, -1))
#inputs = {'input_ids' :inp,'mask':mask}

In [246]:
###ここから違う手法で試している。

In [247]:
tf_batch = tokenizer(test_sentence, max_length = 75, padding=True, truncation=True, return_tensors='tf')

In [248]:
tf_outputs = loaded_model(tf_batch)

In [249]:
tf_outputs

{'logits': <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-1.3125173,  2.463429 , -1.6567228]], dtype=float32)>}

In [250]:
logits = tf_outputs['logits']

# Apply softmax to the logits to get probabilities
probabilities = tf.nn.softmax(logits, axis=-1)
print(probabilities)

# Get the predicted class with the highest probability
predicted_class = np.argmax(probabilities, axis=-1)

# The predicted_class variable now contains the predicted sentiment label (0, 1, or 2)
print("Predicted Sentiment:", predicted_class[0])

tf.Tensor([[0.0220519  0.9623181  0.01563002]], shape=(1, 3), dtype=float32)
Predicted Sentiment: 1


In [191]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define a function to predict sentiment
def predict_sentiment(input_sentence):
    # Tokenize the input sentence
    input_ids = tokenizer.encode(input_sentence, add_special_tokens=True)

    # Pad or truncate the input sequence to a fixed length
    max_length = 128  # You can adjust this based on your model's input size
    input_ids = input_ids[:max_length] + [0] * (max_length - len(input_ids))

    # Create attention mask
    attention_mask = [1 if token_id != 0 else 0 for token_id in input_ids]

    # Convert to tensors
    input_ids = tf.constant([input_ids])
    attention_mask = tf.constant([attention_mask])

    # Make predictions
    predictions = loaded_model.predict([input_ids, attention_mask])

    # Get the predicted class (sentiment label)
    predicted_class = tf.argmax(predictions, axis=1).numpy()[0]

    return predicted_class

# Example usage:
input_sentence = "I really enjoyed the movie. It was fantastic!"
predicted_sentiment = predict_sentiment(input_sentence)

# The predicted_sentiment will be 0, 1, or 2 representing the sentiment label.
print("Predicted Sentiment:", predicted_sentiment)



ValueError: ignored

In [ ]:
#this is the code from hugging face document
input_ids = tf.constant(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True))[None, :]  # Batch size 1
outputs = loaded_model(input_ids)
logits = outputs[0]

ValueError: ignored

In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
loaded_model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (Custom>TFBertMainLay  multiple                  109482240 
 er)                                                             
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
Total params: 109484547 (417.65 MB)
Trainable params: 109484547 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask,request,render_template
import pickle
import numpy as np

In [ ]:
from flask import Flask, request, render_template

app = Flask(__name__)

# ホームページ
@app.route('/')
def home():
    return render_template('index.html')

# 感情分析
@app.route('/analyze', methods=['POST'])
def analyze():
    text = request.form['text']

    # BERTモデルを使用して感情を予測するコードを追加

    # 予測結果を取得
    sentiment = "Positive"  # ここに感情分析結果を代入

    return render_template('result.html', text=text, sentiment=sentiment)

if __name__ == '__main__':
    app.run(debug=True)
